In [1]:
import geopandas as gpd
import rioxarray
import rasterio
import pandas as pd
from rasterio.features import rasterize

# Load the GeoPackage using geopandas
gdf = gpd.read_file('TerraClimate_PDSI.gpkg')

In [2]:
gdf.head(2)

,PDSI_2020-01,PDSI_2020-02,PDSI_2020-03,PDSI_2020-04,PDSI_2020-05,PDSI_2020-06,PDSI_2020-07,PDSI_2020-08,PDSI_2020-09,PDSI_2020-10,PDSI_2020-11,PDSI_2020-12,geometry
0,"-258,810494","305,1047602","641,4363505","617,7495492","781,8043635","701,4215651","629,0991706","564,4379733","506,4159755","454,3939776","389,1687703","547,2767761","POLYGON ((108.47500 -7.77500, 108.52500 -7.775..."
1,"-205,844527","327,2668547","665,4785952","647,7607024","792,8074853","711,0330684","637,622789","572,2125096","513,3253012","460,4380928","398,7216098","550,0516534","POLYGON ((108.42500 -7.77500, 108.47500 -7.775..."


In [3]:
attributes=['PDSI_2020-01', 'PDSI_2020-02', 'PDSI_2020-03', 'PDSI_2020-04', 'PDSI_2020-05', 'PDSI_2020-06',
       'PDSI_2020-07', 'PDSI_2020-08', 'PDSI_2020-09', 'PDSI_2020-10', 'PDSI_2020-11', 'PDSI_2020-12']
# Ensure the attribute columns are numeric
for attr in attributes:
    gdf[attr] = gdf[attr].astype(str).str.replace(',', '.').astype(float)

In [4]:
gdf.head(2)

,PDSI_2020-01,PDSI_2020-02,PDSI_2020-03,PDSI_2020-04,PDSI_2020-05,PDSI_2020-06,PDSI_2020-07,PDSI_2020-08,PDSI_2020-09,PDSI_2020-10,PDSI_2020-11,PDSI_2020-12,geometry
0,-258.810494,305.104760,641.436351,617.749549,781.804364,701.421565,629.099171,564.437973,506.415976,454.393978,389.16877,547.276776,"POLYGON ((108.47500 -7.77500, 108.52500 -7.775..."
1,-205.844527,327.266855,665.478595,647.760702,792.807485,711.033068,637.622789,572.212510,513.325301,460.438093,398.72161,550.051653,"POLYGON ((108.42500 -7.77500, 108.47500 -7.775..."


In [5]:
# Open the reference TIFF with rioxarray to get its properties
reference_raster = rioxarray.open_rasterio('Curah_Hujan.tif')

# Get the spatial properties from the reference raster
transform = reference_raster.rio.transform()
width, height = reference_raster.rio.width, reference_raster.rio.height
crs = reference_raster.rio.crs

# Prepare the shapes for rasterization
raster_bands = []
for attr in attributes:
    shapes = [(geom, value) for geom, value in zip(gdf.geometry, gdf[attr])]
    raster = rasterize(shapes, out_shape=(height, width), transform=transform)
    raster_bands.append(raster)

# Create a new multi-band raster dataset
with rasterio.open(
    'PDSI.tif',
    'w',
    driver='GTiff',
    height=height,
    width=width,
    count=len(attributes),
    dtype=rasterio.float32,
    crs=crs,
    transform=transform,
) as dst:
    for idx, band in enumerate(raster_bands, start=1):
        dst.write(band.astype(rasterio.float32), idx)